In [96]:
import PyPDF2
import re

In [97]:
file_path = 'plan5.pdf'

pdfFileObj = open(file_path, 'rb') 
        
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

page_num = pdfReader.numPages
pages_text = []
for index in range(page_num):
    page = pdfReader.getPage(index)
    pages_text.append(page.extractText())

# extracting text from page 
#print(info) 
    
# closing the pdf file object 
pdfFileObj.close() 

In [98]:
def get_subjects(text) -> str:
    pattern = r'.*(?=Wykonał:)'
    return re.findall(pattern, text, flags=re.DOTALL)[0]

In [99]:
get_subjects(page_1)

NameError: name 'page_1' is not defined

In [ ]:
pattern = r".*(?=Wykonał:)"
out = None
for match in re.finditer(pattern, page_1, re.DOTALL):
    out = match.group(2)

IndexError: no such group

In [ ]:
def plan_author(text) -> str:
    pattern = r'Dziekan:(.*?)(\w+\s[A-Z][a-z]+)(?=[A-Z])'
    return re.findall(pattern, text, flags=re.DOTALL)[0][-1]

In [ ]:
def get_semestr(text) -> str:
    pattern = r'(=?semestr\s)(\d+)'
    return re.findall(pattern, text, flags=re.DOTALL)[0][-1]

In [ ]:
get_semestr(page_1)

'5'

In [ ]:
plan_author(page_1)

'Mariusz Bodzioch'

In [ ]:
def get_date(text, *, week_name: str) -> str:
    """Get date by given week name"""
    pattern = fr'(=?{week_name}\s)([0-9.]+)'
    return re.findall(pattern, text, flags=re.DOTALL)[0][-1]

In [ ]:
get_date(page_1, week_name='Sobota')

'10.10.2020'

In [ ]:
get_date(page_1, week_name='Niedziela')

'11.10.202013'

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd
import enum
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
import pdfminer

# Open a PDF file.
fp = open('plan5.pdf', 'rb')

# Create a PDF parser object associated with the file object.
parser = PDFParser(fp)

# Create a PDF document object that stores the document structure.
# Password for initialization as 2nd parameter
document = PDFDocument(parser)

# Check if the document allows text extraction. If not, abort.
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()

# Create a PDF device object.
device = PDFDevice(rsrcmgr)

# BEGIN LAYOUT ANALYSIS
# Set parameters for analysis.
laparams = LAParams()

# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)

info = []

def parse_obj(lt_objs, index):

    # loop over the object list
    for obj in lt_objs:

        # if it's a textbox, print text and location
        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            print ("%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.get_text().replace('\n', '_')))
            info.append((obj.bbox[0], obj.bbox[1], obj.get_text().replace('\n', '_'), index))
        # if it's a container, recurse
        elif isinstance(obj, pdfminer.layout.LTFigure):
            parse_obj(obj._objs)

# loop over all pages in the document
for i, page in enumerate(PDFPage.create_pages(document)):

    # read the page into a layout object
    interpreter.process_page(page)
    layout = device.get_result()

    # extract text from this object
    parse_obj(layout._objs, i)

   349,    533, Plan NS sem. zimowy 2021/2022_
   309,    516, Informatyka, NS - I st., rok III, semestr 5_
   381,    488, Sobota 02.10.2021_
    85,    473, 07:00_
   137,    473, 08:00_
   188,    473, 09:00_
   239,    473, 10:00_
   291,    473, 11:00_
   342,    473, 12:00_
   394,    473, 13:00_
   445,    473, 14:00_
   496,    473, 15:00_
   548,    473, 16:00_
   599,    473, 17:00_
   650,    473, 18:00_
   702,    473, 19:00_
   753,    473, 20:00_
    85,    336, 07:00_
   137,    336, 08:00_
   188,    336, 09:00_
   239,    336, 10:00_
   291,    336, 11:00_
   342,    336, 12:00_
   394,    336, 13:00_
   445,    336, 14:00_
   496,    336, 15:00_
   548,    336, 16:00_
   599,    336, 17:00_
   650,    336, 18:00_
   702,    336, 19:00_
   753,    336, 20:00_
   375,    351, Niedziela 03.10.2021_
    25,    412, ISI_
    48,    439, ISI 1_
    48,    385, ISI 2_
    25,    275, ISI_
    48,    301, ISI 1_
    48,    248, ISI 2_
    47,    186, ME - Mieernictwo elektron

In [115]:
def get_hour_from_cord_tekst(val: str) -> str:
    if len(hour_definition[hour_definition.x <= val]):
        return hour_definition[hour_definition.x <= val].iloc[-1].Przedmiot
    return -1
def get_hour_from_cord_x(val: str) -> str:
    if len(hour_definition[hour_definition.x <= val]):
        return hour_definition[hour_definition.x <= val].iloc[-1].x
    return -1
def map_hour(hour: str, diff: int):
    if len(str(hour)) < 3:
        return -1
    hour = int(hour[:2])
    if diff > 38:
        return hour + 1
    if diff < 13:
        return f'{hour}:15'
    if diff < 26:
        return f'{hour}:30'
    return f'{hour}:45'

arr = np.array(info)
df = pd.DataFrame(arr)
df.columns = ['x', 'y', 'Przedmiot', 'Strona']
df.x = df.x.astype('float')
df.y = df.y.astype('float')
df

hour_definition = df[(df['Strona'] == '1') & (df['Przedmiot'].str.contains('00_'))]
hour_definition.drop_duplicates(subset=['Przedmiot'], inplace=True)
hour_definition.reset_index(drop=True, inplace=True)
hour_definition

c:\Users\Krystian\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y,Przedmiot,Strona
0,85.80,473.5816,07:00_,1
1,137.18,473.5816,08:00_,1
2,188.54,473.5816,09:00_,1
3,239.90,473.5816,10:00_,1
4,291.29,473.5816,11:00_,1
5,342.65,473.5816,12:00_,1
6,394.01,473.5816,13:00_,1
7,445.39,473.5816,14:00_,1
8,496.75,473.5816,15:00_,1
9,548.11,473.5816,16:00_,1


In [116]:
df_tmp = df[df['Strona'] == '1']
df_tmp['hour_mapped'] = list(map(get_hour_from_cord_tekst, list(df_tmp.x)))
df_tmp['x_cord_mapped'] = list(map(get_hour_from_cord_x, list(df_tmp.x)))
df_tmp['hour_diff'] = df_tmp.x - df_tmp.x_cord_mapped
df_tmp['Czas rozpoczęcia'] = df_tmp.apply(lambda x: map_hour(x['hour_mapped'], x['hour_diff']), axis=1)
df_tmp[df_tmp.Przedmiot.str.contains('\[') == True]

C:\Users\Krystian\AppData\Local\Temp/ipykernel_4468/24079259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['hour_mapped'] = list(map(get_hour_from_cord_tekst, list(df_tmp.x)))
C:\Users\Krystian\AppData\Local\Temp/ipykernel_4468/24079259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['x_cord_mapped'] = list(map(get_hour_from_cord_x, list(df_tmp.x)))
C:\Users\Krystian\AppData\Local\Temp/ipykernel_4468/24079259.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,x,y,Przedmiot,Strona,hour_mapped,x_cord_mapped,hour_diff,Czas rozpoczęcia
60,406.49,424.5316,"ME [w], dr hab. _",1,13:00_,394.01,12.48,13:15
61,496.39,424.5316,"ME [w], dr hab. _",1,14:00_,445.39,51.00,15
66,579.31,409.1716,"WdGM [w], dr hab. _A.Denisiuk,sala C2_",1,16:00_,548.11,31.20,16:45
67,669.22,409.1716,"WdGM [w], dr hab. _A.Denisiuk,sala C2_",1,18:00_,650.86,18.36,18:30
83,130.82,282.0616,"IO [w], dr M.Kolev, _",1,07:00_,85.80,45.02,8
84,220.70,282.0616,"IO [w], dr M.Kolev, _",1,09:00_,188.54,32.16,9:45
87,316.61,287.2216,"ME [w], dr hab. _",1,11:00_,291.29,25.32,11:30
88,406.49,287.2216,"ME [w], dr hab. _",1,13:00_,394.01,12.48,13:15
93,491.47,271.8616,"ERI [w], dr inż. _B.Nowak, sala C2_",1,14:00_,445.39,46.08,15
94,581.35,271.8616,"WWW [w], dr inż. _B.Nowak, sala C2_",1,16:00_,548.11,33.24,16:45


In [ ]:
df_tmp.to_csv('output1.csv', index=False)